## Generate a random matrix of array and dot product

In [1]:

import numpy as np
import scipy as sp
import scipy.integrate as integrate
import re
import decimal

# For example, to create an array filled with random values between 0 and 1, use random function.
# This is particularly useful for problems where you need a random state to get started.
# But here the elements will be decimals

A = np.random.rand(2,3)
print(A)

""" Output of above
[[0.62857547 0.14598864 0.93991874]
 [0.65560569 0.29442998 0.3354452 ]] """


A = np.random.random([2,3]) # Or this is also equivalent
print(A)

'''
numpy.random.random like many of the other numpy.random methods accept shapes, i.e. N-tuples. So really the outside parenthesis represent calling the method numpy.random.random(), and the inside parenthesis are syntactic sugar for instantiating the tuple (3, 3) that is passed into the function.
'''

# Create Matrix (2,3) with random between 0 and 1 and INTEGER (i.e. NOT decimal)
B = np.random.randint(10, size=(3,2))
# print(B)
print(B.T)

# Dot product
print(np.dot(A, B))

# transpose
# print(np.dot(A, B).T)


[[0.31564009 0.67863093 0.11853704]
 [0.66255683 0.95660533 0.77539798]]
[[0.07217951 0.25865577 0.60806084]
 [0.89102432 0.77668313 0.63493438]]
[[3 8 3]
 [8 7 5]]
[[ 4.10996725  5.42833072]
 [10.79134116 15.73964839]]


---

## Integrate curve from vectorized data with numpy.trapz()

First find out the indices of x array that are within the above integral limits:

In [ ]:
# Here are my x-values:

x = [2600, 2601, 2602, 2603, 2604, 2605, 2606, 2607, 2607, 2608, 2609, 2610, 2611, 2612, 2613, 2614, 2615, 2616, 2617, 2618, 2619, 2620, 2621, 2622, 2623, 2624, 2625, 2626, 2627, 2628, 2995, 2996, 2997, 2998, 2999, 3000, 3001, 3002, 3003, 3004, 3005, 3006, 3007, 3008, 3009, 3010, 3011, 3012, 3013, 3014]

# and y-values:

y = [9, 3, 8, 6, 2, 0, 3, 2, 7, 0, 9, 5, 1, 7, 2, 9, 5, 5, 9, 2, 7, 1,
       6, 8, 0, 3, 4, 6, 0, 8, 7, 5, 8, 3, 6, 6, 4, 6, 8, 1, 3, 1, 0, 1,
       8, 9, 0, 1, 1, 3]

# How would you integrate the curve from x = 2672.6 to 3005.3 ?

# Now extract the ranges
idx = np.where(( np.array(x) >= 2672.6 ) & (np.array(x) <= 3005.3))[0]

# And then either using np.trapz to integrate using the trapezoidal rule:
np.trapz(x = np.array(x)[idx], y = np.array(y)[idx] )

# or I could also use from scipy's integration methods for integrating with fixed samples:
# https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html


# same as numpy.trapz
integrate.trapz(x=np.array(x)[idx],y=np.array(y)[idx])
# example using simpson's rule
integrate.simps(x=np.array(x)[idx],y=np.array(y)[idx])

## A note on numpy.trapz() function

#### numpy.trapz() function integrate along the given axis using the composite [trapezoidal rule](https://en.wikipedia.org/wiki/Trapezoidal_rule).

![Imgur](https://imgur.com/kKX9G5R.png)

![Imgur](https://imgur.com/6zwXG8w.png)

You specify the integration range when you pass the x array to the np.trapz function.


Internally, trapz uses np.diff() to calculate dx. This makes the assumption that the x values are increasing monotonically. This leads to wrong values when x does not have this property:

```python
x = array([0, 1, 2])

y = array([0, 2, 4])

trapz(y, x) # 4, correct value

trapz(y[::-1], x[::-1]) # -4, incorrect value

idx = random.permutation(arange(3))

x_shuffled = x[idx]

y_shuffled = y[idx]

trapz(y_shuffled, x_shuffled) # 1, incorrect value
```

This limitation of trapz can be fixed by sorting both y, x values before proceeding.

Numpy repo's maintainers have explicitly retained this, opining this as a feature rather than a defect. See [here](https://github.com/numpy/numpy/issues/4601#issuecomment-88697254)


"It seems nearly all of the scipy community sees the sensitivity to x-array ordering as a feature rather than a defect. While I believe this position has an equally valid point of view (the disagreement is over who has to keep track of their minus signs explicitly) it seems that changing the behavior will cause far more upset than molification."

### When can I get a negative value from numpy.trapz()

#### If the $x_i$ in the call to trapz() are not in increasing order, then you will get negative results. So if you want a positive number from this function when it returns a negative, Try re-ordering x in ascending order (and y-values accordingly):

```python
x_order = x(end:-1:1); %fliplr
y_order = y(end:-1:1); %fliplr
trapz(x_order, y_order)
```

 In essence, I will get negative result, when I am integrating the function represented by the y data from x = 1 to x = 0, not from x = 0 to x = 1. If I flip my x vector so that I am integrating from x = 0 to x = 1 (essentially swapping the limits of integration) then the area will be positive.

In `trapz(x,y)` differentiation of x is applied through diff(x,1,1), i.e. `[x(2:n,:) - x(1:n-1,:)].` If your x is descending this will give negative dx. It doesn't matter if it is positive or negative. However, in `plot` the curve will appear positive-definite (you don't actually see the order of points, just pairs from two vectors on a plane).

**Example** (compare the following):

```python
x = [-1 -0.5 0]; y = 0.5-x;
figure; plot(x,y); hold on; plot(-x, y,'r')
trapz(x, y)
trapz(-x, y)
figure; plot(x, y); hold on; plot(fliplr(-x), fliplr(y),'r')
trapz(fliplr(-x), fliplr(y))
```


Think of it like this. The integral of a function that is always positive, if the limits are inverted, will still be negative. Thus we know that

    int(x^2,-1,1) = 2/3

But

    int(x^2,1,-1) = -2/3

Clearly x^2 is always a positive number, but here the limits of integration were not increasing, but decreasing.

If the $x_i$ in the call to trapz are not in increasing order, then you will get negative results. This is reflected by trapz. Trapz sees the order of the points as presented to it.

    x = -1:.1:1;
    trapz(x,x.^2)
    ans =
             0.67

    xrev = fliplr(x);
    trapz(xrev,xrev.^2)
    ans =
            -0.67

The plot shows only that the function is positive, not the order of the points.

---

## Considering a four dimensions array, how to get sum over the last two axis at once?

First a quick refresher on axis in Numpy

### Axis 0 will act on all the ROWS in each COLUMN

### Axis 1 will act on all the COLUMNS in each ROW

### So a `mean` on axis 0 will be the mean of all the rows in each column, and a mean on axis 1 will be a mean of all the columns in each row.


In [2]:
x = np.array([[1, 2, 3], [4,5,6],[7,8,9]], np.int32)
print(x)
'''
array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]], dtype=int32)
'''

s0 = x.sum(axis=0)  # sum the columns [1,4,7] = 12, [2,5,8] = 15 [3,6,9] = 18
s0

[[1 2 3]
 [4 5 6]
 [7 8 9]]


array([12, 15, 18])

In [3]:
s1 = x.sum(axis=1)    # sum the rows [1,2,3] = 6, [4,5,6] = 15 [7,8,9] = 24
s1

array([ 6, 15, 24])

### sum() function for a three dimensional array:

In [4]:
x = np.array((((1,2), (3,4) ), ((5,6),(7,8))))
x

'''
  array([[[1, 2],
          [3, 4]],
         [[5, 6],
          [7, 8]]])
'''

'\n  array([[[1, 2],\n          [3, 4]],\n         [[5, 6],\n          [7, 8]]])\n'

In [5]:
x.shape # dimensions of the array
# (2, 2, 2)

x.sum(axis=0)

'''
array([[ 6,  8],   #  [1,5] = 6 [2,6] = 8 [3,7] = 10 [4, 8] = 12
      [10, 12]])
'''
x.sum(axis=1)

'''
array([[ 4,  6],   # [1,3] = 4 [2,4] = 6 [5, 7] = 12 [6, 8] = 14
      [12, 14]])
'''

x.sum(axis=2) # [1, 2] = 3 [3, 4] = 7 [5, 6] = 11 [7, 8] = 15
'''
array([[ 3,  7],
      [11, 15]])
'''

x.ndim # number of dimensions of the array
# 3

3

In [6]:
# Now finally, considering a four dimensions array, how to get sum over the last two axis at once?

array_4_d = np.random.randint(0, 10, (2, 4, 2, 4))
print(array_4_d)
print('shape of above array_4_d ', array_4_d.shape)

"""
Syntax of random.randint : numpy.random.randint(low, high=None, size=None, dtype=’l’)

 It returns an array of specified shape and fills it with random integers from low (inclusive) to high (exclusive), i.e. in the interval [low, high).

size is the third optional parameter - in above its withing the parenthesis. It takes a size int or tuple of ints.

This determines output shape. If the given shape is, e.g., (m, n, k), then m * n * k samples are drawn. Default is None, in which case a single value is returned.

"""

# Solution-alternate-1 - by passing a tuple of axes

sum_1 = array_4_d.sum(axis=(-2, -1))
sum_1

[[[[7 9 7 2]
   [4 6 8 2]]

  [[5 9 2 3]
   [1 9 2 7]]

  [[3 6 1 4]
   [9 3 1 1]]

  [[2 3 6 3]
   [5 2 9 0]]]


 [[[4 0 2 8]
   [6 1 7 5]]

  [[8 6 8 6]
   [3 4 7 2]]

  [[1 9 3 1]
   [4 3 3 0]]

  [[8 2 3 6]
   [8 8 4 3]]]]
shape of above array_4_d  (2, 4, 2, 4)


array([[45, 38, 28, 30],
       [33, 44, 24, 42]])

In [7]:
# Solution alternative 2 - by flattening the last two dimensions into one
# (useful for functions that don't accept tuples for axis argument)

sum_2 = array_4_d.reshape(array_4_d.shape[:-2] + (-1,)).sum(axis=-1)
sum_2


array([[45, 38, 28, 30],
       [33, 44, 24, 42]])

---

# Numpy Concatenate vs hstack

In [8]:
a = np.array([1, 2,3])
b = np.array([5, 6,7])

z = np.concatenate((a, b))
# In above line, need to pass the arrays as an iterable (a tuple or list)
# otherwise will be below error
# TypeError: only integer scalar arrays can be converted to a scalar index 

c = np.stack([a,b]).reshape(-1)
d = np.hstack([a,b])

print(z)
print(c)
print(d)
# all three will print [1 2 3 5 6 7]

[1 2 3 5 6 7]
[1 2 3 5 6 7]
[1 2 3 5 6 7]


### Numpy Concatenate 2D arrays

![Imgur](https://imgur.com/n5pmWGo.png)

In [9]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])

x = np.concatenate((a, b), axis=0)
print(x)
'''
[[1 2]
 [3 4]
 [5 6]]
'''


[[1 2]
 [3 4]
 [5 6]]


'\n[[1 2]\n [3 4]\n [5 6]]\n'

---

### numpy vstack

It Stack arrays in sequence vertically i.e row wise

You can concatenate the above 2D arrays using vstack and will get the same result as concatenate with axis =0

In [10]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])

v = np.vstack((a, b))
print(v)

'''
[[1 2]
 [3 4]
 [5 6]]
'''

[[1 2]
 [3 4]
 [5 6]]


'\n[[1 2]\n [3 4]\n [5 6]]\n'

### Let’s merge the above two arrays along the axis =1

`y = np.concatenate((a, b), axis=1)`

Output: ValueError: all the input array dimensions for the concatenation axis must match exactly

But why it’s throwing an error, because both the arrays doesn’t have the same dimensions along 0 to concatenate

So how do we change the dimension for this concatenation

Transpose the array B and then concatenate along axis = 1

![Imgur](https://imgur.com/AvZEtPK.png)



In [11]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])

y = np.concatenate((a, b.T), axis=1)
print(y)

'''
[[1 2 5]
 [3 4 6]]
'''

[[1 2 5]
 [3 4 6]]


'\n[[1 2 5]\n [3 4 6]]\n'

## The reshape() function in Numpy

#### The reshape() function is used to give a new shape to an array without changing its data.

`numpy.reshape(a, newshape, order='C')`

[Doc](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html#numpy-reshape)

#### newshape	The new shape should be compatible with the original shape. If an integer, then the result will be a 1-D array of that length. One shape dimension can be -1. In this case, the value is inferred from the length of the array and remaining dimensions.

The below example

![Imgur](https://imgur.com/fbjyxqd.png)


In [12]:
x = np.array([[2,3,4], [5,6,7]])      
np.reshape(x, (3, 2))

array([[2, 3],
       [4, 5],
       [6, 7]])

### What does -1 mean in numpy reshape() ?

The "-1" stands for "unknown dimension" which can and should be infered from another dimension.

#### Main Rule is - when we don't how many columns the resultant matrix should have (set it to -1!), but if we know that we want a 1-dimensional array(set the first parameter to 1!).


In [13]:
a = np.array([[2,3,4], [5,6,7]])

print(np.reshape(a, 6))

print(np.reshape(a, 6, order='F'))

print(np.reshape(a, (3,-1)))
# the unspecified value is inferred to be 2

[2 3 4 5 6 7]
[2 5 3 6 4 7]
[[2 3]
 [4 5]
 [6 7]]


#### In the last line above, we wanted a 3 rows, but we did not know how many columns the resultant matrix should have, and so we kept it at -1. Hence numpy decided the unspecified value - which was inferred to be 2

In [14]:
a = np.matrix([[1, 2, 3, 4], [5, 6, 7, 8]])
b = np.reshape(a, -1)
b

matrix([[1, 2, 3, 4, 5, 6, 7, 8]])

### numpy allow us to give one of new shape parameter as -1 (eg: (2,-1) or (-1,3) but not (-1, -1)). It simply means that it is an unknown dimension and we want numpy to figure it out. And numpy will figure this by looking at the 'length of the array and remaining dimensions' and making sure it satisfies the above mentioned criteria


for the below example the output explains the resultant vector to be a single row.(-1) indicates the number of rows to be 1.
if the 

    a = numpy.matrix([[1, 2, 3, 4], [5, 6, 7, 8]])
    b = numpy.reshape(a, -1)

output:

    matrix([[1, 2, 3, 4, 5, 6, 7, 8]])

 The "-1" stands for "unknown dimension" which can should be infered from another dimension.


In [15]:
a = np.matrix([[1, 2, 3, 4], [5, 6, 7, 8]])
b = np.reshape(a, (1, -1))
b

matrix([[1, 2, 3, 4, 5, 6, 7, 8]])

But 

```python
a = np.matrix([[1, 2, 3, 4], [5, 6, 7, 8]])
b = np.reshape(a, (-1, 1))
```

Gives a column vector

In [16]:
a = np.matrix([[1, 2, 3, 4], [5, 6, 7, 8]])
b = np.reshape(a, (-1, 1))
b

matrix([[1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8]])

But 

```python
a = np.matrix([[1, 2, 3, 4], [5, 6, 7, 8]])
b = np.reshape(a, (-1, 1))
```

Gives a column vector. 

#### Trying to reshape with (-1, 1) . We have provided column as 1 but rows as unknown . So we get result new shape as (8, 1).again compatible with original shape(2,4)

#### The above is consistent with numpy advice/error message, to use reshape(-1,1) for a single feature; i.e. single column

Reshape your data using `array.reshape(-1, 1)` if your data has a single feature

In [17]:
a = np.matrix([[1, 2, 3, 4], [5, 6, 7, 8]])
b = np.reshape(a, (-1, 1))
b

matrix([[1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8]])

And finally, if we try to provide both dimension as unknown i.e new shape as (-1,-1). It will throw an error

```python
a.reshape(-1, -1)
ValueError: can only specify one unknown dimension
```

## Reshaping Numpy Array Numerical Column/Features before Normalization

##### The main Principle that I am going to discuss with example below is this

### For normalizing numerical data, we got to use reshape(1, -1) and NOT reshape(-1, 1) and then apply Normalize() function. Ofcourse after the Normalization is done we can rehape it back with (-1, 1) if thats what we need.

Here is how normalizer works and why you should use reshape(1, -1) instead of (-1, 1)

### Normalizer by default normalizes on each sample(row) while StandardScaler standardises on each feature(column)

---

### What does -1 mean in numpy reshape() ?

The "-1" stands for "unknown dimension" which can and should be infered from another dimension.

#### Main Rule is - when we don't how many columns the resultant matrix should have (set it to -1!), but if we know that we want a 1-dimensional array(set the first parameter to 1!).

---

Lets take an example where my array is `np.array([1, 2, 3])`

### Noting the mechanism of reshape() - that numpy allow us to give one of new shape parameter as -1 (eg: (2,-1) or (-1,3) but not (-1, -1)). It simply means that it is an unknown dimension and we want numpy to figure it out. And numpy will figure this by looking at the 'length of the array and remaining dimensions' and making sure it satisfies the above mentioned criteria

Here if I use (-1, 1) it means any number of rows, which is the responsibility of numpy to figure out, while I am specifying that I need to have one column. 

Rememeber -1 lets numpy to calculate the unknown dimension for the resultant that will match with the original matrix.

And vice versa, if I do reshape(1, -1) means, that I am specifying row to be 1 while leaving column numbers to be calculated by Numpy.

So for the case, that I use (-1, 1) => i.e. Rows are unknows while columns required is 1

![Imgur](https://imgur.com/vv8tAQK.png)

Normalizer() will go each sample wise i.e. row wise and calculate the Normalizer formulae for that row.


Lets do actual example

In [18]:
from sklearn.preprocessing import Normalizer
import numpy as np

a = np.array([1, 2, 3])

b = np.reshape(a, (-1, 1))

c = np.reshape(a, (1, -1))

print('Afer reshaping with (-1, 1) \n', b)
print('After reshaping with (1, -1) \n', c)

Afer reshaping with (-1, 1) 
 [[1]
 [2]
 [3]]
After reshaping with (1, -1) 
 [[1 2 3]]


In [19]:
normalizer = Normalizer()

a_transformed_not_useful = normalizer.transform(b)
c_transformed = normalizer.transform(c)

print("a_transformed_not_useful \n \n", a_transformed_not_useful)
print("c_transformed \n \n", c_transformed)

a_transformed_not_useful 
 
 [[1.]
 [1.]
 [1.]]
c_transformed 
 
 [[0.26726124 0.53452248 0.80178373]]


As we can see the result of the reshape `a_transformed_not_useful` is a column vector with each value of 1 which in most cases is not useful for our modelling. Beause Normalizer() will go each sample wise i.e. row wise and calculate the Normalizer formulae for that row.

---

## How argmax() function works in Numpy

### argmax is a function which gives the index of the greatest number in the given row or column and the row or column can be decided using axis attribute of argmax funcion. 

### If we give axis=0 then the function will now try to find the maximum value along the rows of the matrix. And  give the index from rows. If the maximum value is in the 2-nd row (i.e. index of 1), then 1 will be returned.

### If we give axis=1 then it will give the index from column. If the maximum value is in the 3rd column, index 2 then that index number will be returned.



In [20]:

A = np.matrix([[]])

A = np.matrix([[1, 2, 3, 33], [4, 5, 6, 66], [7, 8, 9, 99]])

print(np.argmax(A))
# 11, which is the position of 99

print(np.argmax(A[:, :]))
# 11, which is the position of 99

print(np.argmax(A[:1]))
# 3, which is the position of 33

print(np.argmax(A[:, 2]))
# 2, which is the position of 9

np.argmax(A[1:, 2])
# 1, which is the position of 9
""" A[1:, 2] it will first fetch the values from 1st row on wards and the only 2nd column value from those rows, then it will find the index of max value from into the resulted matrix. """


a = [[1, 2, 3], [4, 5, 6]]
print(np.argmax(a))
"""
# 5

the array is 2 dimensional, with shape (2,3). Since no axis parameter is specified in the function, the numpy library flattens the array to a 1 dimensional array and then returns the index of the maximum value. In this case the array is transformed to [[1,2,3,4,5,6]] and then returns the index of 6, which is 5.

"""

11
11
3
2
5


'\n# 5\n\nthe array is 2 dimensional, with shape (2,3). Since no axis parameter is specified in the function, the numpy library flattens the array to a 1 dimensional array and then returns the index of the maximum value. In this case the array is transformed to [[1,2,3,4,5,6]] and then returns the index of 6, which is 5.\n\n'

#### When parameter is axis = 0 in argmax()

In [21]:

a = [[1,2,3],[4,5,6]]
np.argmax(a, axis=0)
# array([1, 1, 1])

array([1, 1, 1])

The result here is a bit confusing at first. 

#### First repeating the original rule -  

### If we give axis=0 then the function will now try to find the maximum value along the rows of the matrix. And  give the index from rows. If the maximum value is in the 2-nd row (i.e. index of 1), then 1 will be returned.

### If we give axis=1 then it will give the index from column. If the maximum value is in the 3rd column, index 2 then that index number will be returned.

Since the axis is defined to be 0, the function will now try to find the maximum value along the rows of the matrix. 

The maximum value, 6, is in the second row of the matrix. The index of the second row is 1. According to the documentation https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.argmax.html the dimension specified in the axis parameter will be removed. Since the shape of the original matrix was (2,3) and axis specified as 0, the returned matrix will have a shape of(3,) instead, since the 2 in the original shape(2,3) is removed.The row in which the maximum value was found is now repeated for the same number of elements as the columns in the original matrix i.e. 3.


### When parameter is axis = 1 in argmax()

In [22]:
a = [[1,2,3],[4,5,6]]
np.argmax(a, axis=1)
# array([2, 2])

array([2, 2])

Same concept as above but now index of the column is returned at which the maximum value is available. In this example the maximum value 6 is in the 3rd column, index 2. The column of the original matrix with shape (2,3) will be removed, transforming to (2,) and so the return array will display two elements, each showing the index of the column in which the maximum value was found.